<a href="https://colab.research.google.com/github/PapaZeusOne/Automated-Trading-Bot/blob/main/Automating_Stock_Trades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [135]:
import numpy as np 
import pandas as pd
import os, io
import datetime as dt
import pandas_datareader.data as web
import requests

# **TODO**


> Vergleiche mehrere Aktien miteinander



> Sende Push zu der Aktie, mit der besten Bewertung



> Prüfen wie stark der CCI gewachsen ist in den letzten 14 Tagen

# **Weitere Ideen**


> Aktien Sektoren zuordnen und den Bot nach Sektoren abfragen. Der Bot soll dann die stärkste Aktie aus dem Sektor zurückgeben. Ziemlich cool, wenn man weiß das zum Beispiel der Wasserstoffsektor gerade extrem am steigen ist.




> Sobald man sich für eine Aktie entschieden, wäre es ja auch interessant zu wissen, wann man am besten aussteigen sollte. Dann könnte man Abfragen schreiben und über den Bot eine Push rausschicken, ob man eine bestimmte Aktie, die man mit dem Bot teilt, verkaufen soll oder nicht, weil sich gerade der Kurs schlecht entwickelt.





# **Indikatoren**


> **MACD & Signallinie**: Der MACD (Moving Average Convergence/Divergence) wird in der Chartanalyse genutzt um Trends sowie Kauf-und Verkaufssignale zu erkennen. Dabei werden zwei exponentiell gewichtete gleitende Durchschnitte (26 und 12 Tage) berechnet und voneinander subtrahiert. Des Weiteren wird eine Signallinie berechnet. Hierbei handelt es sich um den 9 Tage exponentiell geglätteten Durchschnitt. Ein steigender MACD ist ein Indikator für einen Aufwärtstrend, ein sinkender MACD ein Indikator für einen Abwärtstrend. Schneidet der MACD die Signallinie von unten nach oben, so liegt ein Kaufsignal vor. Wird die Signallinie vom MACD von oben nach unten geschnitten, so liegt ein Verkaufssignal vor.



> **RSI**: Der RSI (Relative-Stärke-Index) ist ein Marktindikator. Er wird berechnet, indem der gleitende Mittelwert der Auf-und Abwärtsveränderungen für einen bestimmten Zeitraum (in der Regel 14 Tage) ermittelt wird. Der RSI kann Werte zwischen 0 und 100 einnehmen. Die Werte 30 und 70 spielen beim RSI eine besondere Rolle: Bei einem Wert unter 30 gilt der Wert als überverkauft. Dies bedeutet, dass eine Kurserholung (Steigerung) in nächster Zeit anstehen könnte. Bei einem Wert über 70 gilt die Aktie als überkauft. Dies ist ein Indiz dafür, dass der Kurs in nächster Zeit fallen könnte.




> **Bollinger Band**: Es existieren eigentlich zwei Bollinger Bänder, die gemeinsam mit dem gleitenden Mittel, dem arithmetischen Mittel der Kurse, dann das sogenannte Bollinger Band bilden. Mit Hilfe des Bollinger Bands werden der Verlauf der letzten Tage eines Wertpapieres und dessen Trend visualisiert. Man sieht, wo der Kurs durchschnittlich lag und wie stark er an welchen Tagen vom Mittelwert abwich. In der Regel verwendet man dafür den Zeitraum der letzten 20 Tage.





> **Bollinger Bands oder Keltner Channels?** Schau hier warum die BB die bessere Wahl wären: https://tradeciety.com/keltner-channel-vs-bollinger-bands-one-better/ Aber statt der Verwendung der Standardabweichung zur Darstellung der Bollinger Bänder wird die Average True Range von Wilders eingesetzt. Dieser Unterschied bewirkt, dass der Keltner Channel ruhiger und gleichmäßiger aussieht als die Bollinger Bänder.



> **EMA**: Exponential Moving Average verwendet man, um Kursschwankungen zu glätten und den zugrunde liegenden Trend dadurch besser zu erkennen. Hierzu betrachtet man einen Kursverlauf für einen bestimmten Zeitraum in der Vergangenheit und berechnet daraus den Durchschnitt. **Formel**: https://admiralmarkets.com/de/wissen/articles/forex-indicators/ema-der-exponentiell-gleitende-durchschnitt-als-universelles-hilfsmittel-fur-ihr-trading




> **Commodity Channel Index (CCI)**: Für den CCI wird die Abweichung zwischen Kurs und dessen Gleitenden Durchschnitt herangezogen, um das Momentum zu erfassen. Um dies zu bewerkstelligen, wird die Abweichung zwischen dem aktuellen Betrachtungszeitraum und der durchschnittlichen Abweichung der vergangenen X Zeitperioden verglichen. Im Gegensatz zum Stochastik-Oszillator und dem RSI sind die Werte des CCI nicht begrenzt. Aber in der Praxis bewegt sich der CCI aufgrund seiner Konstruktion zwischen -100 und +100. 
Bemerkenswerterweise werden beim CCI nicht die Schlusskurse, sondern die typischen Kurse (Hoch + Tief + Schlusskurs / 3) zugrunde gelegt. Dann wird die Abweichung bzw. Entfernung zwischen typischem Kurs und Gleitendem Durchschnitt ermittelt. **Formel**: https://www.kagels-trading.de/technische-indikatoren-trader/ **WICHTIG**: Wie der Name schon sagt, gründet der CCI auf der Annahme, dass sich die Kurse normalerweise innerhalb eines unsichtbaren Kanals um den Gleitenden Durchschnitt herum aufhalten.














# **Indikatoren mit denen wir anfangen sollten**


> 1. EMA


> 2. MACD


> 3. Bollinger Bänder



> 4. RSI












In [136]:
#indicator = pd.DataFrame(df, index=df.index)

In [137]:
def calc_indicators(df):
  df['ema_25'] = calc_ema_25(df)
  df['ema_100'] = calc_ema_100(df)

  df['sma_25'] = calc_sma_25(df)
  df['sma_100'] = calc_sma_100(df)

  calc_bollinger(df)

  calc_macd_signalline(df)

  df['RSI'] = calc_rsi(df)

  df['CCI'] = calc_cci(df)

  # df['OBV_MA'] = calc_obv(df)
   
  df = calculate_dead_cat_bounce_v2(df)

  return df

In [138]:
def calc_ema_25(df):
    ema_25 = 25
    df_ema_25 = df['Adj Close'].ewm(span=ema_25, adjust=False).mean()

    return df_ema_25

def calc_ema_100(df):
    ema_100 = 100
    df_ema_100 = df['Adj Close'].ewm(span=ema_100, adjust=False).mean()

    return df_ema_100

In [139]:
def calc_sma_25(df):
    sma_25 = 25
    df_sma_25 = df['Adj Close'].rolling(window=sma_25).mean()

    return df_sma_25

def calc_sma_100(df):
    sma_100 = 100
    df_sma_100 = df['Adj Close'].rolling(window=sma_100).mean()

    return df_sma_100

In [140]:
def calc_bollinger(df):
    bollinger_avg = 20

    short_avg_ma = df['Adj Close'].rolling(window=bollinger_avg).mean()
    short_avg_std = df['Adj Close'].rolling(window=bollinger_avg).std()

    df['bollinger'] = df['Adj Close'].ewm(span=bollinger_avg, adjust=False).mean()
    df['bollinger_lower_band'] = short_avg_ma - (short_avg_std *2)
    df['bollinger_upper_band'] = short_avg_ma + (short_avg_std *2)

In [141]:
def calc_macd_signalline(df):
    ema_short = 12
    ema_long = 26
    signalline_span = 9

    df_ema_short = df['Adj Close'].ewm(span=ema_short, adjust=False).mean()
    df_ema_long = df['Adj Close'].ewm(span=ema_long, adjust=False).mean()
    df['macd'] = df_ema_short - df_ema_long
    df['signalline'] = df['macd'].ewm(span=signalline_span, adjust=False).mean()

In [142]:
def calc_rsi(df, n=14):
    delta = df['Adj Close'].diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    # RolUp = pd.rolling_mean(dUp, n)
    # RolDown = pd.rolling_mean(dDown, n).abs()

    # Calculate the EWMA
    RolUp = dUp.ewm(span=n).mean()
    RolDown = dDown.abs().ewm(span=n).mean()

    # Calculate the RSI based on EWMA
    RS = RolUp / RolDown
    RSI = 100 - (100/(1+RS))

    # Calculate the SMA
    roll_up2 = dUp.rolling(n).mean()
    roll_down2 = dDown.abs().rolling(n).mean()

    # Calculate the RSI based on SMA
    RS2 = roll_up2 / roll_down2
    RSI2 = 100.0 - (100.0 / (1.0 + RS2))

    own_RSI=(RSI+RSI2)/2
    return own_RSI

In [143]:
def calc_cci(df, n=14, constant=0.015):
    tippingPrice = (df['High'] + df['Low'] + df['Adj Close']) / 3
    tippingPrice_mean = tippingPrice.rolling(n).mean()
    tippingPrice_std = tippingPrice.rolling(n).std()
    
    CCI = pd.Series((tippingPrice - tippingPrice_mean) / (constant * tippingPrice_std)) 
    return CCI

In [144]:
### To Be Adjusted

def calc_obv(df, n = 20):
    i = 0
    OBV = [0]

    while i < df.index[-1]:
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] > 0:
            OBV.append(df.loc[i + 1, 'Volume'])
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] == 0:
            OBV.append(0)
        if df.loc[i + 1, 'Adj Close'] - df.loc[i, 'Adj Close'] < 0:
            OBV.append(-df.loc[i + 1, 'Volume'])
        i = i + 1
    OBV = pd.Series(OBV)
    OBV_ma = pd.Series(OBV.rolling(n, min_periods=n).mean())

    return OBV_ma

In [145]:
def calculate_dead_cat_bounce_v2(df):
  set_freq = '2D'

  df['body_mid'] = (df['Adj Close'] + df['Open'])/2

  pct_change_close = df['body_mid'].pct_change(freq=set_freq, fill_method='ffill')
  pct_change_close.dropna(inplace=True)

  dead_cat_bounce = np.where((df.signalline > df.macd) &
                            ((pct_change_close >= 0.05) | (pct_change_close <= -0.05)) &
                            ((df.Open < df.bollinger_lower_band)|(df.Close < df.bollinger_lower_band)),
                            True, False)

  df['dead_cat_bounce'] = dead_cat_bounce
  del df['body_mid']

  return df

In [146]:
def combine_stocks_to_dataframe(id_list):
  start = dt.datetime(2019,1,1)
  end = dt.datetime.now()
  #end = dt.datetime(2017,1,1)
  df_list = []

  for id in id_list:
    stock = web.DataReader(id, 'yahoo', start, end) # load data
    stock['stock_id'] = id
    stock = calc_indicators(stock) # calc indicators for every stock
    stock.dropna(inplace=True) # maybe delete this later
    df_list.append(stock)

  df_complete = pd.concat(df_list)
  
  return df_complete

In [147]:
df_complete = combine_stocks_to_dataframe(['AMZN', 'AAPL', 'SAP', 'ACN', 'VOW.DE', 'PLUG', 'MDO.DE', 'PYPL', 'SQ', 'V', 'YNDX'])

In [148]:
df_yd = df_complete.drop_duplicates(subset='stock_id', keep='last')

In [149]:
def compute_score_v2(row):
  score=0
  if row['ema_25']>row['ema_100']:
    score+=1
  if row['macd']>row['signalline']:
    score+=1
  if row['Adj Close']>row['bollinger_upper_band']:
    score-=1
  if ((row['RSI']>=50) & (row['RSI']<=70)) | (row['RSI'] <= 30):
    score+=1
  if row['Adj Close']<row['bollinger_lower_band']:
    score+=1
  if row['CCI']<-100:
      score+=1
  if row['Adj Close']>row['sma_25']:
      score+=1
  return score

In [150]:
def ema_score(row):
    ema_25 = row['ema_25']
    ema_100 = row['ema_100']
    score = (ema_25/ema_100)-1
    # if score < 0:
    #     score=0
    return score

def macd_signalline_score(row): # need to change something here
    # relative change might be distorted if values near by 0
    macd = row['macd']
    signalline = row['signalline']
    adj_close = row['Adj Close']
    score = (macd/adj_close+1)-(signalline/adj_close+1)
    return score

# set weighting when close is lower lower_band or upper upper_band
# to be discussed
def bollinger_score(row):
    lower_band = row['bollinger_lower_band']
    upper_band = row['bollinger_upper_band']
    middle_band = row['bollinger']
    adj_close = row['Adj Close']
    if (adj_close>=middle_band):
        if adj_close>=upper_band:
            score = ((adj_close/upper_band)-1)*(-1)
        else:
            score = ((adj_close/((upper_band-middle_band)*0.5+middle_band))-1)*(-1)
    else:
        if (adj_close<=lower_band):
            score = ((adj_close/lower_band)-1)*(-1)
        else:
            score = (adj_close/middle_band)-1
    return score

def sma_score(row):
    sma_25 = row['sma_25']
    adj_close = row['Adj Close']
    score = (adj_close/sma_25)-1
    return score

def rsi_score(row):
    lower_band = 30
    upper_band = 70
    middle_band = (lower_band + upper_band) / 2 
    middle_band1 = 55
    middle_band2 = 45
    outlier_factor = 1.2
    rsi = row['RSI']
    if (rsi>middle_band):
        if rsi>=upper_band:
            score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi<middle_band1:
                score = 0
            else:
                if rsi<=62.5:
                    score = rsi/middle_band1-1
                else:
                    score = ((62.5-(rsi-62.5))/(middle_band1))-1
    else:
        if (rsi<=lower_band):
            score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
        else:
            if rsi>middle_band2:
                score = 0
            else:
                if rsi>=37.5:
                    score = (rsi/middle_band2-1)
                else:
                    score = (37.5-(rsi-37.5))/middle_band2-1
    return score

# We should check whether we give the same weighting if it hits above or lower 100/-100
def cci_score(row):
    lower_band = -100
    middle_lower_band = -50
    upper_band = 105
    middle_band = 0
    middle_upper_band = 85
    outlier_factor = 0.25
    outlier_factor1 = 2
    cci=row['RSI']

    if (cci < lower_band):
        score=(cci/lower_band)-1
    else:
        if (cci<=middle_lower_band):
            score = (cci / lower_band)-1
        elif (cci<middle_band):
            score = (((cci-lower_band)/lower_band)+1)*(-1)
        else:
            if (cci>upper_band):
                score = ((cci/upper_band)-1)*(-1)
            else:
                if(cci<=middle_upper_band):
                    score = (cci/upper_band)*outlier_factor
                else:
                    if(cci<upper_band):
                        score=(middle_upper_band-(cci-middle_upper_band))/upper_band-(middle_upper_band-(middle_upper_band-upper_band))/middle_upper_band
                    else:
                        score=0
    
    return score

In [153]:
def relative_score(row):
    r = row
    stock = row['stock_id']
    ema_scr = ema_score(r)
    macd_signalline_scr = macd_signalline_score(r)
    bollinger_scr = bollinger_score(r)
    sma_scr = sma_score(r)
    rsi_scr = rsi_score(r)
    cci_scr = cci_score(r)

    score = ema_scr + (macd_signalline_scr*5) + bollinger_scr + sma_scr + rsi_scr + cci_scr
    print(stock, ema_scr, (macd_signalline_scr*5), bollinger_scr, sma_scr, rsi_scr, cci_scr, score, '\n')
    return score

In [154]:
df_yd['score'] = df_yd.apply(lambda row: relative_score(row), axis=1)
df_yd.sort_values(by=['score'], ascending=False)

AMZN 0.007450680462110837 -0.004719562524909127 -0.017266796876552526 -0.02243250949506559 -0.001465510712959528 0.1069858381378972 0.06855213899052126 

AAPL 0.002350632578016265 -0.00745879749266265 -0.021260665250036115 -0.02637686948603357 -0.0959931850008322 0.09685787303562512 -0.05188101161592315 

SAP 0.0413009271542617 -0.012314629778721331 -0.005459303585114927 -0.0037899397658668343 0 0.11883054216645567 0.13856759619101428 

ACN 0.04393107659570994 -0.014290750547092945 -0.014411572872837564 -0.021941685765041186 -0.141515245746757 0.09198050938427603 -0.05624766895174273 

VOW.DE 0.1411874669855886 0.07424048864302368 -0.031705811127211714 0.12811424704378238 -0.1931543995432478 0.19349366660322884 0.31217565860516394 

PLUG -0.14166763835827545 0.13210243153473744 0.022660030499769057 0.133909312922891 -0.09077298844124479 0.1792740261723951 0.23550517433027235 

MDO.DE 0.028180605397315484 0.005861495385676707 -0.0005815669242501542 0.005835423296788322 0.089352851364082

,High,Low,Open,Close,Volume,Adj Close,stock_id,ema_25,ema_100,sma_25,sma_100,bollinger,bollinger_lower_band,bollinger_upper_band,macd,signalline,RSI,CCI,dead_cat_bounce,score
Date,,,,,,,,,,,,,,,,,,,,
2021-06-03,260.980011,256.149994,258.399994,257.790009,4943300.0,257.790009,PYPL,255.480398,248.847090,252.869601,255.852851,255.659346,236.901219,266.593783,1.260744,-0.168885,62.880518,41.059152,False,0.365781
2021-06-04,312.000000,309.200012,310.000000,310.799988,15583.0,310.799988,VOW.DE,284.180505,249.021754,275.504000,243.065001,286.656843,241.699532,315.840467,9.592189,4.977400,81.267340,102.200424,False,0.312176
2021-06-03,67.209999,66.669998,67.190002,66.750000,784000.0,66.750000,YNDX,65.496868,64.749879,65.294001,65.457000,65.782761,61.369089,69.169912,1.004618,0.790712,65.333121,36.991428,False,0.301030
2021-06-04,192.100006,191.600006,192.050003,192.050003,179.0,192.050003,MDO.DE,190.548961,185.326352,190.935812,182.885787,190.627354,187.208729,193.249402,0.182953,-0.042187,59.914407,88.986239,False,0.271302
2021-06-03,32.020000,29.809999,30.670000,30.030001,29589600.0,30.030001,PLUG,28.206500,32.861978,26.483600,41.656600,28.200605,19.727088,33.251912,0.642313,-0.151095,75.295091,76.535881,False,0.235505
2021-06-03,230.389999,227.770004,229.000000,228.110001,6381500.0,228.110001,V,226.960955,219.305373,228.036575,216.987243,227.273468,221.770067,232.063240,0.687654,0.738155,57.450341,50.911175,False,0.222247
2021-06-03,138.399994,137.479996,137.889999,137.839996,682700.0,137.839996,SAP,138.113436,132.635468,138.364389,129.871861,138.596637,134.266488,142.813705,1.111985,1.451474,49.908828,-62.299609,False,0.138568
2021-06-03,219.940002,211.250000,218.869995,211.429993,7231500.0,211.429993,SQ,220.022639,224.080769,218.526799,233.420099,218.333163,193.123120,233.261879,-4.032796,-6.012808,51.339826,19.666070,False,0.086859
2021-06-03,3214.439941,3184.030029,3204.229980,3187.010010,2393500.0,3187.010010,AMZN,3247.835457,3223.815835,3260.143203,3224.568191,3243.006342,3151.542250,3308.052745,-18.133512,-15.125254,44.934052,-122.472057,False,0.068552


In [19]:
lower_band = 30
upper_band = 70
middle_band = (lower_band + upper_band) / 2 
middle_band1 = 55
middle_band2 = 45
outlier_factor = 1.2
rsi = 73
if (rsi>middle_band):
    if rsi>=upper_band:
        score = ((rsi/upper_band)-1)*(-1)*(outlier_factor)
    else:
        if rsi<middle_band1:
            score = 0
        else:
            if rsi<=62.5:
                score = rsi/middle_band1-1
            else:
                score = ((62.5-(rsi-62.5))/(middle_band1))-1
else:
    if (rsi<=lower_band):
        score = ((rsi/lower_band)-1)*(-1)*(outlier_factor)
    else:
        if rsi>middle_band2:
            score = 0
        else:
            if rsi>=37.5:
                score = (rsi/middle_band2-1)
            else:
                score = (37.5-(rsi-37.5))/middle_band2-1
score

-0.05142857142857151

In [20]:
# df_yd['score'] = df_yd.apply(lambda row: compute_score_v2(row), axis=1)
# df_yd

In [21]:
#df_complete[df_complete.dead_cat_bounce==True]

In [22]:
def bot_warn_me(text):
    
    # Change the strings: <TOKEN> and <CHAT_ID>, with your token and chat_id
    bot_token_id = '1790740607:AAE7Omfuea3tG3iii9A1qX-vQJ9pdLFcO0s'
    bot_chat_id = '908941210'
    
    send_text = 'https://api.telegram.org/bot' + bot_token_id + '/sendMessage?chat_id=' + bot_chat_id + '&parse_mode=Markdown&text=' + text
    
    response = requests.get(send_text)

    return response.json()